In [1]:
%matplotlib inline

In [2]:
import fatiando
import numpy as np
from IPython.display import Image
from fatiando.vis import mpl, myv
import datetime
import string as st

In [3]:
mpl.rcParams['font.size'] = 16

In [4]:
import functions as af

In [5]:
notebook_name = 'application_real_data_PUC_2.ipynb'

In [6]:
print "Version of Fatiando a Terra used: {}".format(fatiando.__version__)

Version of Fatiando a Terra used: 0.4


In [7]:
saved_files = [] ### List of saved files

### Load data from txt files

In [8]:
files = ['..\\data\\data0_2.txt', 
         '..\\data\\data1_2.txt',
         '..\\data\\data2_2.txt',
         '..\\data\\data3_2.txt']

nfiles = len(files)

In [9]:
data = []

for i in range(nfiles):
    data.append(np.loadtxt(files[i]))

### Parameters of the sample

In [10]:
N = 16 # number of prisms forming the sample
M = 3*N # number of parameters to be estimated

Lx = 0.001 # in m
Ly = 0.003 # in m
Lz = Ly  # in m

shape = (42,102) 
#shape = (102,42)

In [11]:
xmin_sample = -0.5*Lx*N #in m
centers = [100.*(xmin_sample + 0.5*Lx + i*Lx) for i in range(N)] #in cm
centers = np.array(centers)

### Interpretation model

In [12]:
model = af.sample(Lx,Ly,Lz,N)

In [13]:
xmin = np.min([data[0][:,0], data[1][:,0], data[2][:,0], data[3][:,0]])
xmax = np.max([data[0][:,0], data[1][:,0], data[2][:,0], data[3][:,0]])
ymin = np.min([data[0][:,1], data[1][:,1], data[2][:,1], data[3][:,1]])
ymax = np.max([data[0][:,1], data[1][:,1], data[2][:,1], data[3][:,1]])
zmin = np.min([data[0][:,2], data[1][:,2], data[2][:,2], data[3][:,2]])
zmax = np.max([data[0][:,2], data[1][:,2], data[2][:,2], data[3][:,2]])

volume = [xmin, xmax, ymin, ymax, zmin, zmax]
area = [xmin,xmax,ymin,ymax]

In [14]:
scene = myv.figure()
myv.prisms(model, color= (0.4,0.4,0.4))

myv.points(np.vstack((data[0][:,0], data[0][:,1], data[0][:,2])).T, 
           color=(1, 0, 0), size=0.00005, opacity=1, spherical=False)
myv.points(np.vstack((data[1][:,0], data[1][:,1], data[1][:,2])).T, 
           color=(0, 0, 1), size=0.00005, opacity=1, spherical=False)
myv.points(np.vstack((data[2][:,0], data[2][:,1], data[2][:,2])).T, 
           color=(0, 1, 0), size=0.00005, opacity=1, spherical=False)
myv.points(np.vstack((data[3][:,0], data[3][:,1], data[3][:,2])).T, 
           color=(0, 0, 0), size=0.00005, opacity=1, spherical=False)



ax = myv.axes(myv.outline(volume), ranges=[b for b in volume], nlabels=5, fmt='%.3f')
ax.axes.x_label, ax.axes.y_label, ax.axes.z_label = 'x (cm)', 'y (cm)', 'z (cm)'
ax.axes.font_factor = 1.2
myv.wall_bottom(ax.axes.bounds, opacity=0.2)
myv.wall_north(ax.axes.bounds)
#myv.savefig('model2.png')
myv.show()
#Image(filename='model2.png')

### Inversion

In [15]:
B_obs = []
for i in range(4):
    B_obs.append(data[i][:,3])

In [16]:
A = []
for i in range(4):
    A.append(af.sensitivity(N, data[i][:,0], data[i][:,1], data[i][:,2], model,i))

In [17]:
R = np.zeros(M+1)
R[0] = 1.
R[3] = -1.
R = np.resize(R, (M-3,M))

In [18]:
u0 = 0.

In [19]:
H = np.dot(np.vstack(A).T, np.vstack(A))

In [20]:
f0 = np.trace(H)/M
f0

12526138.706979195

In [21]:
H = H + u0*f0*np.dot(R.T, R)

In [22]:
h = np.dot(np.vstack(A).T, np.hstack(B_obs))

In [23]:
p_est = np.linalg.solve(H, h)

In [24]:
p_est = af.L2_norm(np.vstack(A),np.hstack(B_obs))
#p_est = f.L2_norm(np.vstack((A[1],A[2],A[3])),np.hstack((B_obs[1],B_obs[2], B_obs[3])))
#p_est = f.L2_norm(np.vstack((A[0],A[1],A[2])),np.hstack((B_obs[0],B_obs[1], B_obs[2])))
#p_est = f.L2_norm(np.vstack((A[1],A[2])),np.hstack((B_obs[1],B_obs[2])))
#p_est = f.L2_norm(np.array(A[1]),np.array(B_obs[1]))
#p_est = f.L2_norm(np.array(A[0]),np.array(B_obs[0]))

In [25]:
mag_r = af.parameters_sph(N,p_est)

In [30]:
#Save the total-field anomaly in a txt file
now = datetime.datetime.utcnow().strftime('%d %B %Y %H:%M:%S UTC')
header = """# Generated by {name} on {date}
#
# Parameters estimated through inversion using the method proposed in this work.
# The result is the magnetization component within each prism that compose the interpretative model.
# 
# Regular grid with
# shape (nx, ny): {shape}
# area [xmin, xmax, ymin, ymax]: area
# 
# Parameters:
#
# Intensity (A/m)    Inclination (degree)   Declination (degree): 
#
#
""".format(name=notebook_name, date=now, shape = 'shape', area = 'area')
file_name = '..\\data\\estimate_real_2.txt'
for i in range(N):
    with open(file_name, 'w') as f:
        f.write(header)
        np.savetxt(f, mag_r, fmt='%.5f')
    saved_files.append(file_name)

In [28]:
çççççççççççççççççççççççççççççççççç

SyntaxError: invalid syntax (<ipython-input-28-a8b9a2673cdc>, line 1)

In [ ]:
np.savetxt('..\\data\\estimate_real_2.txt', np.array(mag_r), fmt='%.5f')

In [ ]:
B_pre = []
for i in range(4):
    B_pre.append(np.dot(A[i],p_est))

In [ ]:
r_norm = []
r_mean = []
r_std = []

for i in range(4):
    aux1,aux2,aux3 = f.residual(0.001*B_obs[i],0.001*B_pre[i])
    r_norm.append(aux1) # in uT
    r_mean.append(aux2) # in uT
    r_std.append(aux3)  # in uT

### Plotting of the observed and predicted data 

In [ ]:
title_font = 22
bottom_font = 16

labels = ['(a)', '(b)', '(c)', 
          '(d)', '(e)', '(f)',
          '(g)', '(h)', '(i)',
          '(j)', '(k)', '(l)']

lables_place = (0.05, 0.90)
lables_size = 24

mpl.close('all')
mpl.figure(figsize=(20,24), tight_layout=True)

for i in range(0,4,2):

    # in uT
    ranges = 0.001*np.abs([B_obs[i].max(), B_obs[i].min(),
                           B_pre[i].max(), B_pre[i].min()]).max()

    mpl.subplot(4,3,1 + 3*i)

    #mpl.title(labels[3*i], fontsize=title_font)
    nlevels = mpl.contourf(100.*data[i][:,1], 100.*data[i][:,0], 0.001*B_obs[i], 
                           shape, 20, cmap=mpl.cm.RdBu_r,
                           vmin=-ranges, vmax=ranges)
    #mpl.colorbar(pad=0.01, aspect=40, shrink=1.0).set_label('uT')
    mpl.colorbar(pad=0.01, aspect=40, shrink=1.0)
    mpl.xlabel('y (cm)', fontsize = title_font)
    mpl.ylabel('x (cm)', fontsize = title_font)
    mpl.annotate(labels[3*i], xy = lables_place, 
                 xycoords = 'axes fraction', fontsize=lables_size)

    mpl.subplot(4,3,1 + 3*i + 1)

    #mpl.title(labels[3*i + 1], fontsize=title_font)
    nlevels = mpl.contourf(100.*data[i][:,1], 100.*data[i][:,0], 0.001*B_pre[i], 
                           shape, 20, cmap=mpl.cm.RdBu_r,
                           vmin=-ranges, vmax=ranges)
    #mpl.colorbar(pad=0.01, aspect=40, shrink=1.0).set_label('uT')
    mpl.colorbar(pad=0.01, aspect=40, shrink=1.0)
    mpl.xlabel('y (cm)', fontsize = title_font)
    mpl.ylabel('x (cm)', fontsize = title_font)
    mpl.annotate(labels[3*i + 1], xy = lables_place, 
                 xycoords = 'axes fraction', fontsize=lables_size)

    mpl.subplot(4,3,1 + 3*i + 2)
    #mpl.title(labels[3*i + 2], fontsize=title_font)
    mpl.xlabel('$\mu$ = %.3f uT | $\sigma$ = %.3f uT' % (r_mean[i], r_std[i]), fontsize = title_font)
    nbins = int((np.max(r_norm[i]) - np.min(r_norm[i])))
    mpl.hist(r_norm[i],bins=nbins,normed=True)
    mpl.ylim(0.,0.5)
    mpl.xlim(-10., 10.)
    mpl.annotate(labels[3*i + 2], xy = lables_place, 
                 xycoords = 'axes fraction', fontsize=lables_size)
    
for i in range(1,4,2):

    # in uT
    ranges = 0.001*np.abs([B_obs[i].max(), B_obs[i].min(),
                           B_pre[i].max(), B_pre[i].min()]).max()

    mpl.subplot(4,3,1 + 3*i)

    #mpl.title(labels[3*i], fontsize=title_font)
    nlevels = mpl.contourf(100.*data[i][:,2], 100.*data[i][:,0], 0.001*B_obs[i], 
                           shape, 20, cmap=mpl.cm.RdBu_r,
                           vmin=-ranges, vmax=ranges)
    #mpl.colorbar(pad=0.01, aspect=40, shrink=1.0).set_label('uT')
    mpl.colorbar(pad=0.01, aspect=40, shrink=1.0)
    mpl.xlabel('z (cm)', fontsize = title_font)
    mpl.ylabel('x (cm)', fontsize = title_font)
    mpl.annotate(labels[3*i], xy = lables_place, 
                 xycoords = 'axes fraction', fontsize=lables_size)

    mpl.subplot(4,3,1 + 3*i + 1)

    #mpl.title(labels[3*i + 1], fontsize=title_font)
    nlevels = mpl.contourf(100.*data[i][:,2], 100.*data[i][:,0], 0.001*B_pre[i], 
                           shape, 20, cmap=mpl.cm.RdBu_r,
                           vmin=-ranges, vmax=ranges)
    #mpl.colorbar(pad=0.01, aspect=40, shrink=1.0).set_label('uT')
    mpl.colorbar(pad=0.01, aspect=40, shrink=1.0)
    mpl.xlabel('z (cm)', fontsize = title_font)
    mpl.ylabel('x (cm)', fontsize = title_font)
    mpl.annotate(labels[3*i+1], xy = lables_place, 
                 xycoords = 'axes fraction', fontsize=lables_size)

    mpl.subplot(4,3,1 + 3*i + 2)
    #mpl.title(labels[3*i + 2], fontsize=title_font)
    mpl.xlabel('$\mu$ = %.3f uT | $\sigma$ = %.3f uT' % (r_mean[i], r_std[i]), fontsize = title_font)
    nbins = int((np.max(r_norm[i]) - np.min(r_norm[i])))
    mpl.hist(r_norm[i],bins=nbins,normed=True)
    mpl.ylim(0.,0.5)
    mpl.xlim(-10., 10.)
    mpl.annotate(labels[3*i+2], xy = lables_place, 
                 xycoords = 'axes fraction', fontsize=lables_size)

mpl.savefig('..\\manuscript\\Figs\\datafit_real_2.png')
mpl.show()

In [ ]:
mpl.figure(figsize=(8,8))
ax = mpl.subplot(111, polar=True)
ax.set_theta_direction(-1) 
ax.set_theta_zero_location('N') 
ax.set_rgrids([0.2,0.4,0.6,0.8], angle=90.)

f.proj_polar(4,mag_r[:4,:],color='r')
f.proj_polar(4,mag_r[4:8,:],color='g')
f.proj_polar(4,mag_r[8:12,:],color='b')
f.proj_polar(4,mag_r[12:16,:],color='k')

mpl.xticks(fontsize=20) 
mpl.yticks(fontsize=14) 
mpl.ylim(0.0, 1.0)
#mpl.savefig('estimate_polar_real.png')
mpl.show() 

In [ ]:
non_spurious = np.array([True for i in range(N)])
non_spurious[::4] = False

spurious = np.array([False for i in range(N)])
spurious[::4] = True

In [ ]:
label_font = 20
legend_font = 16

labels = ['(a)', '(b)', '(c)']

lables_place = (0.03, 0.90)
lables_size = 24

mpl.show()
mpl.close('all')
mpl.figure(figsize=(8, 16), tight_layout=True)

mpl.subplot(3,1,1)
mpl.plot([-400.*Lx, -400.*Lx], [0., 1500.0], 'k-')
mpl.plot([0., 0.], [0., 1500.0], 'k-')
mpl.plot([400.*Lx, 400.*Lx], [0., 1500.0], 'k-')
mpl.plot([np.min(centers), np.max(centers)], [0.0, 0.0], 'k--')
#mpl.plot([np.min(centers), np.max(centers)], [1200.0, 1200.0], 'k--',
#         label='Saturation magnetization')
mpl.plot(centers, mag_r[:,0], 'b--', linewidth=2)
mpl.plot(centers[non_spurious], mag_r[non_spurious,0], 'bo', markersize=10)
mpl.plot(centers[spurious], mag_r[spurious,0], 'k^', markersize=12)
mpl.ylabel('Intensity ($A/m$)', fontsize=label_font)
mpl.ylim(0., np.max(mag_r[:,0]) + 50.)
#mpl.legend(loc='best', numpoints=1)
mpl.annotate(labels[0], xy = lables_place, 
                 xycoords = 'axes fraction', fontsize=lables_size)
mpl.annotate('0', xy = (0.12, 0.5), 
                 xycoords = 'axes fraction', fontsize=18)
mpl.annotate('1', xy = (0.37, 0.5), 
                 xycoords = 'axes fraction', fontsize=18)
mpl.annotate('2', xy = (0.62, 0.5), 
                 xycoords = 'axes fraction', fontsize=18)
mpl.annotate('3', xy = (0.87, 0.5), 
                 xycoords = 'axes fraction', fontsize=18)

mpl.subplot(3,1,2)
mpl.plot([-400.*Lx, -400.*Lx], [-180., 180.0], 'k-')
mpl.plot([0., 0.], [-180., 180.0], 'k-')
mpl.plot([400.*Lx, 400.*Lx], [-180., 180.0], 'k-')
mpl.plot([np.min(centers), np.max(centers)], [0.0, 0.0], 'k--')
mpl.plot([np.min(centers), np.max(centers)], [45.0, 45.0], 'k--')
mpl.plot([np.min(centers), np.max(centers)], [-90.0, -90.0], 'k--')
mpl.plot([np.min(centers), np.max(centers)], [90.0, 90.0], 'k--')
#mpl.plot([np.min(centers), np.max(centers)], [45.0, 45.0], 'k--')
#mpl.plot([np.min(centers), np.max(centers)], [90.0, 90.0], 'k--')
#mpl.plot([np.min(centers), np.max(centers)], [-90.0, -90.0], 'k--')
mpl.plot(centers, mag_r[:,1], 'b--', linewidth=2)
mpl.plot(centers[non_spurious], mag_r[non_spurious,1], 'bo', markersize=10)
mpl.plot(centers[spurious], mag_r[spurious,1], 'k^', markersize=12)
mpl.ylabel('Inclination ($^{\circ}$)', fontsize=label_font)
mpl.ylim(np.min(mag_r[:,1]) - 20., np.max(mag_r[:,1]) + 20.)
#mpl.legend(loc='best', numpoints=1)
mpl.annotate(labels[1], xy = lables_place, 
                 xycoords = 'axes fraction', fontsize=lables_size,
                 backgroundcolor = 'w')
mpl.annotate('0', xy = (0.12, 0.5), 
                 xycoords = 'axes fraction', fontsize=18)
mpl.annotate('1', xy = (0.37, 0.5), 
                 xycoords = 'axes fraction', fontsize=18)
mpl.annotate('2', xy = (0.62, 0.5), 
                 xycoords = 'axes fraction', fontsize=18)
mpl.annotate('3', xy = (0.87, 0.5), 
                 xycoords = 'axes fraction', fontsize=18)

mpl.subplot(3,1,3)
mpl.plot([-400.*Lx, -400.*Lx], [-180., 200.0], 'k-')
mpl.plot([0., 0.], [-180., 200.0], 'k-')
mpl.plot([400.*Lx, 400.*Lx], [-180., 200.0], 'k-')
mpl.plot([np.min(centers), np.max(centers)], [0.0, 0.0], 'k--')
mpl.plot([np.min(centers), np.max(centers)], [180.0, 180.0], 'k--')
mpl.plot([np.min(centers), np.max(centers)], [-180.0, -180.0], 'k--')
#mpl.plot([np.min(centers), np.max(centers)], [180.0, 180.0], 'k--')
mpl.plot(centers, mag_r[:,2], 'b--', linewidth=2)
mpl.plot(centers[non_spurious], mag_r[non_spurious,2], 'bo', markersize=10)
mpl.plot(centers[spurious], mag_r[spurious,2], 'k^', markersize=12)
mpl.ylabel('Declination ($^{\circ}$)', fontsize=label_font)
#mpl.xlabel('Sample length ($cm$)', fontsize=label_font)
mpl.xlabel('$x$ ($cm$)', fontsize=label_font)
mpl.ylim(np.min(mag_r[:,2]) - 20., np.max(mag_r[:,2]) + 20.)
#mpl.savefig('estimate_int_inc_dec_simul_validation.eps')
#mpl.legend(loc='best', numpoints=1)
mpl.annotate(labels[2], xy = (0.03, 0.05), 
                 xycoords = 'axes fraction', fontsize=lables_size,
                 backgroundcolor = 'w')
mpl.annotate('0', xy = (0.12, 0.5), 
                 xycoords = 'axes fraction', fontsize=18)
mpl.annotate('1', xy = (0.37, 0.5), 
                 xycoords = 'axes fraction', fontsize=18)
mpl.annotate('2', xy = (0.62, 0.5), 
                 xycoords = 'axes fraction', fontsize=18)
mpl.annotate('3', xy = (0.87, 0.5), 
                 xycoords = 'axes fraction', fontsize=18)

mpl.savefig('..\\manuscript\\Figs\\estimate_int_inc_dec_real_2.png')

mpl.show()

### Discussion

The synthetic sample is formed by four justaposed cubes along the x diretion. The interpretation model used in this simulation is formed by 16 prisms, so that each cube of the original synthetic sample is described by four prisms. 